<a href="https://colab.research.google.com/github/SofiaGori/API/blob/main/OpenFIGI_corp_bonds/All_corp_bonds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **OpenFIGI**

*Sofia Gori, Jan 2021*

This codes retrieves all securities ever issued by the firms in our sample.

The companies in our sample have been chosen because they received an ESG disclosure score from Bloomberg.

**Housekeeping**

In [ ]:
import ast
import json         
import pprint
import requests  
import openpyxl
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

OpenFIGI request - example

In [ ]:
jobs = [
    {'idType': 'TICKER', 'idValue': 'ADS', 'micCode': 'XETR'},
    {'idType': 'ID_ISIN', 'idValue': 'XS2265968284'},  
    {'idType': 'BASE_TICKER', 'idValue': 'ZIGGO','securityType2':'Corp'}
]

print(jobs)

<class 'list'>


**Import tickers from Excel** 

Upload the file in Google CoLab

In [ ]:
from google.colab import files
uploaded = files.upload()

#Please, import a file of your choosing containing tickers:

Saving Prova.xlsx to Prova.xlsx


Select the tickers column and take only the unique values

In [ ]:
excel = pd.read_excel('Prova.xlsx')

ticker = pd.DataFrame(excel['Co. Corp TKR'].unique())
ticker.columns = ['ticker']
display(ticker)

,ticker
0,STERV
1,ADRIT
2,FSBIOE
3,FLUVIU
4,ZIGGO
...,...
826,CYPRSB
827,GXBBGB
828,RIZHBK
829,AEONRE


**API setup**

In [ ]:
openfigi_apikey  = '2900b1fe-2eb2-4504-a9e9-bb8bc81b26b3'  #API Key
openfigi_url     = 'https://api.openfigi.com/v2/mapping'
openfigi_headers = {'Content-Type': 'text/json'}

**API query function definition**

In [ ]:
def map_securities(jobs):
    if openfigi_apikey:
        openfigi_headers['X-OPENFIGI-APIKEY'] = openfigi_apikey
    response = requests.post(url=openfigi_url, headers=openfigi_headers,
                             json=jobs)
    if response.status_code != 200:
        raise Exception('Bad response code {}'.format(str(response.status_code)))
    return response.json()

**Function to transform the query from JSON format into a table in panel data format**

In [ ]:
def map_table(query):
  
  #First JSON normalization: remove 'error' node and leave 'data' node.
  normalization = pd.json_normalize(query)
  normalization = normalization[normalization['data'].notna()]
  normalization.index = range(len(normalization))
  normalization = normalization['data']

  #Run the second JSON normalization for each company.
  #Create a database for each company that contains all securities ever issued.
  output = pd.DataFrame()

  for i in range(len(normalization)):
    norm = pd.json_normalize(normalization[i])
    output = output.append(norm, ignore_index=True)
  
  return output

**Query (in loop)**

Loop setup

In [ ]:
# Create an empty dataframe
database = pd.DataFrame()

# first and last are non-changing parts of the query
first = "{'idType': 'BASE_TICKER', 'idValue': '"
last  = "', 'securityType2' : 'Corp'}"

Run the loop (one ticker at a time)

In [ ]:
  for tck in ticker.ticker[x:x+10]:
    query = []
    query_list = ast.literal_eval(first + tck + last)
    query.append(query_list)
    print(query)
    
    result = map_securities(query)
    data   = map_table(result)
    
    database = database.append(data, ignore_index=True)

**Edit the database**

Create a column that retrieves the maturity of every security from its security ticker

In [ ]:
database['ticker_corp'], database['num']= database['ticker'].str.split(' ', 1).str
database['num_letter'], database['date']= database['num'].str.split(' ', 1).str
database['maturity'], database['rest']= database['date'].str.split(' ', 1).str
database.drop(['num', 'num_letter', 'date', 'rest'], axis=1, inplace=True)

database.set_index('ticker_corp',inplace=True)
database.reset_index(inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  This is separate from the ipykernel package so we can avoid doing imports until


Display the database as a table

In [ ]:
print("\n--------------\nStart database\n--------------\n")
display(database)
print("\n--------------\nEnd database\n--------------\n")


--------------
Start database
--------------



,ticker_corp,figi,name,ticker,exchCode,compositeFIGI,uniqueID,securityType,marketSector,shareClassFIGI,uniqueIDFutOpt,securityType2,securityDescription,maturity
0,STERV,BBG00001Y7S3,STORA ENSO OYJ,STERV 7.375 05/15/11,NOT LISTED,None,COEC3932547,GLOBAL,Corp,None,None,Corp,STERV 7 3/8 05/15/11,05/15/11
1,STERV,BBG000045ZM3,STORA ENSO OYJ,STERV 6.375 06/29/07 EMTN,FRANKFURT,None,COEC2659885,EURO MTN,Corp,None,None,Corp,STERV 6 3/8 06/29/07,06/29/07
2,STERV,BBG00004V274,STORA ENSO OYJ,STERV F 10/11/10 EMTN,LUXEMBOURG,None,COEC2972767,EURO MTN,Corp,None,None,Corp,STERV Float 10/11/10,10/11/10
3,STERV,BBG000079X56,STORA ENSO OYJ,STERV 3.5 12/01/15 CPI,LUXEMBOURG,None,COEF3081645,EURO MTN,Corp,None,None,Corp,STERV 3 1/2 12/01/15,12/01/15
4,STERV,BBG000088H23,STORA ENSO OYJ,STERV 5.125 06/23/14 EMTN,FRANKFURT,None,COED5148983,EURO MTN,Corp,None,None,Corp,STERV 5 1/8 06/23/14,06/23/14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1851027,HANMIL,BBG00YS7NBF3,KB CAPITAL CO LTD,HANMIL 1.698 01/08/26 4656,NOT LISTED,None,COBN3901087,DOMESTIC,Corp,None,None,Corp,HANMIL 1.698 01/08/26,01/08/26
1851028,HANMIL,BBG00Z0JRX08,KB CAPITAL CO LTD,HANMIL 1.219 07/25/23 4661,NOT LISTED,None,COBN6945552,DOMESTIC,Corp,None,None,Corp,HANMIL 1.219 07/25/23,07/25/23
1851029,HANMIL,BBG00Z0JSCB1,KB CAPITAL CO LTD,HANMIL 1.295 01/25/24 4662,NOT LISTED,None,COBN6945784,DOMESTIC,Corp,None,None,Corp,HANMIL 1.295 01/25/24,01/25/24
1851030,HANMIL,BBG00Z0JW5W8,KB CAPITAL CO LTD,HANMIL 1.307 02/23/24 4663,NOT LISTED,None,COBN6948382,DOMESTIC,Corp,None,None,Corp,HANMIL 1.307 02/23/24,02/23/24



--------------
End database
--------------



**Export the database**

Make sure you can download the files from Google CoLab

In [ ]:
from google.colab import files

Export as .csv file

In [ ]:
database.to_csv('all_securities.csv')
files.download('all_securities.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Export as .txt file

In [ ]:
df.to_csv('all_securities.txt', header=True, index=True, sep=',', mode= 'a')
files.download('all_securities.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>